In [1]:
import gym
print(gym.__version__)

0.13.1


In [2]:
import random
import math
import numpy as np
from collections import deque

In [ ]:
# Building the openAI gym

env=gym.make('CartPole-v0')

for i_episode in range(20):
    observation=env.reset()
    for t in range(1000):
        env.render()
#         print(observation)
        action=env.action_space.sample()
        observation, reward, done, info=env.step(action)
        
#         if done:
#             print(f'Eposide finished after {t+1} timesteps')
#             break

That is our enviroment and what we want to is to teach the renforcement network to apply a force to left or to right so that it balances that pole on top

In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


The discreate space allows a fixed range of non-negative numbers and in this case the allowed actions are going to be either 0 or 1 and the box space represents an N dimensional box and valid obsevations for this case wiil be an array of 4 numbers 

In [4]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


This are the possible ranges of the observations, these are probably specifying things like location of the cart, location of the pole, we have two actions here 1 going to be force left or right

## Defining Parameters

In [11]:
n_episodes=1000
n_win_ticks= 195
max_env_steps=None

gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01

batch_size=64
monitor=False
quiet=False

#Enviroment Paramenters
memory=deque(maxlen=100000)
env=gym.make('CartPole-v0')

if max_env_steps is not None:
    env.max_episode_steps=max_env_steps

## Building the Neural Network

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [10]:
model=Sequential()

In [12]:
model.add(Dense(24,input_dim=4,activation='relu'))
model.add(Dense(48,activation='relu'))
model.add(Dense(2,activation='relu'))
model.compile(loss='mse',optimizer=Adam(lr=alpha, decay=alpha_decay))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
#define necessary functions

def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))
    
def choose_action(state,epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(psilon_min,min(epsilon,1.0 - math.logic((t+1)*epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state,[1,4])

def replay(batch_size,epsilon):
    x_batch,y_batch=[],[]
    minibatch=random.sample(memory,min(len(memory),batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        y_target=model.predict(state)
        y_target[0][action]=reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    
    
    if epsilon > epsilon_min:
        epsilon*=epsilon_decay

In [15]:
# The main run function

def run():
    scores=deque(maxlen=100)
    
    for e in range(n_episodes):
        state=preprocess_state(env.reset())
        done=False
        i=0
        while not done:
            action =choose_action(state, get_epsilon(e))
            next_state,reward,done,_=env.step(action)
            env.render()
            next_state=preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i+=1
            
        scores.append(i)
        mean_score=np.mean(scores)
        
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet:
                print(f'Ran {e} episodes. Solved after {e-100} trails')
                return e - 100
            
            if e % 20 == 0 and not quiet:
                print(f'Episodes {e} - mean survival time over last 100 episodes was {mean_score} ticks')
                
                
                
            replay(batch_size,epsilon)
    if not quiet:
        print(f'Did not Solve after {e} wpisodes')
        return e

Just copying everything done above into a single cell

In [25]:
# Training the network

import random
import gym
import math
import numpy as np
from collections import deque

n_episodes=1000
n_win_ticks= 195
max_env_steps=None

gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01

batch_size=64
monitor=False
quiet=False

#Enviroment Paramenters
memory=deque(maxlen=100000)
env=gym.make('CartPole-v0')

if max_env_steps is not None:
    env.max_episode_steps=max_env_steps

from collections import deque    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model=Sequential()
model.add(Dense(24,input_dim=4,activation='relu'))
model.add(Dense(48,activation='relu'))
model.add(Dense(2,activation='relu'))
model.compile(loss='mse',optimizer=Adam(lr=alpha, decay=alpha_decay))

#define necessary functions

def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))
    
def choose_action(state,epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min,min(epsilon,1.0 - math.log10((t+1)*epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state,[1,4])

def replay(batch_size,epsilon):
    x_batch,y_batch=[],[]
    minibatch=random.sample(memory,min(len(memory),batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        y_target=model.predict(state)
        y_target[0][action]=reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    
    
    if epsilon > epsilon_min:
        epsilon*=epsilon_decay

# The main run function

def run():
    scores=deque(maxlen=100)
    
    for e in range(n_episodes):
        state=preprocess_state(env.reset())
        done=False
        i=0
        while not done:
            action =choose_action(state, get_epsilon(e))
            next_state,reward,done,_=env.step(action)
            env.render()
            next_state=preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i+=1
            
        scores.append(i)
        mean_score=np.mean(scores)
        
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet:
                print(f'Ran {e} episodes. Solved after {e-100} trails')
                return e - 100
            
        if e % 20 == 0 and not quiet:
            print(f'Episodes {e} - mean survival time over last 100 episodes was {mean_score} ticks')
                
                
                
            replay(batch_size,get_epsilon(e))
    if not quiet:
        print(f'Did not Solve after {e} wpisodes')
        return e

run()

Episodes 0 - mean survival time over last 100 episodes was 23.0 ticks
Episodes 20 - mean survival time over last 100 episodes was 76.0952380952381 ticks
Episodes 40 - mean survival time over last 100 episodes was 64.58536585365853 ticks
Episodes 60 - mean survival time over last 100 episodes was 61.52459016393443 ticks
Episodes 80 - mean survival time over last 100 episodes was 58.22222222222222 ticks
Episodes 100 - mean survival time over last 100 episodes was 58.39 ticks
Episodes 120 - mean survival time over last 100 episodes was 52.05 ticks
Episodes 140 - mean survival time over last 100 episodes was 50.13 ticks
Episodes 160 - mean survival time over last 100 episodes was 49.2 ticks
Episodes 180 - mean survival time over last 100 episodes was 49.09 ticks
Episodes 200 - mean survival time over last 100 episodes was 49.19 ticks
Episodes 220 - mean survival time over last 100 episodes was 50.3 ticks
Episodes 240 - mean survival time over last 100 episodes was 50.81 ticks
Episodes 260 

999